In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import os

In [2]:
import os
import re
from torchvision.io import read_image, ImageReadMode
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, img_dir, labels, transform=None):
        self.img_dir = img_dir
        self.labels = labels
        self.transform = transform
        self.file_paths = []
        self.label_map = {'not_car': 0, 'car': 1}
        self.classes = labels
        
        for lab in self.labels:
            label_folder = os.path.join(str(img_dir), lab)
            file_list = [os.path.join(label_folder, file.name) for file in os.scandir(label_folder) if file.is_file()]
            self.file_paths.extend([(file, lab) for file in file_list])
    
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        img_path, label = self.file_paths[idx]
        image = read_image(img_path, mode=ImageReadMode.RGB).float() / 255.0 # Scale to [0, 1]
        if self.transform:
            image = self.transform(image)
        label_id = self.label_map[label]
        return image, label_id
        
# Example usage
train_dir = 'D:/MSRSGI/Summer_Semester24/DL/Exercises/Exercise1/WorkingData/data/train'
test_dir = 'D:/MSRSGI/Summer_Semester24/DL/Exercises/Exercise1/WorkingData/data/test'
labels = ['not_car', 'car']  # Ensure this matches your directory structure

# Define the transforms
import torchvision.transforms as transforms

data_transforms = transforms.Compose([
    transforms.Resize((224, 224))
    # transforms.ToTensor()
])

train_dataset = CustomDataset(train_dir, labels, transform = data_transforms) 
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True) #takes an image not a tensor 

test_dataset = CustomDataset(test_dir, labels, transform = data_transforms) 
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=True) #takes an image not a tensor

In [3]:
len(train_dataset), len(test_dataset)

(4200, 1346)

In [4]:
def get_mean_std(loader):
    mean = 0.
    std = 0.
    total_image_count = 0
    for images, _ in loader:
        image_count_in_a_batch = images.size(0)
        images = images.view(image_count_in_a_batch, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_image_count += image_count_in_a_batch
        
    mean /= total_image_count
    std /= total_image_count
    
    return mean, std

# data_transforms = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
# ])

batch_size = 32
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
mean_train, std_train = get_mean_std(train_dataloader)
mean_train, std_train

(tensor([0.4886, 0.4886, 0.4760]), tensor([0.1482, 0.1395, 0.1342]))

In [5]:
batch_size = 32
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
mean_test, std_test = get_mean_std(test_dataloader)
mean_test, std_test

(tensor([0.4670, 0.4365, 0.4319]), tensor([0.1659, 0.1498, 0.1380]))

In [6]:
# Define data transformations for data augmentation and normalization
train_transforms_norm = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.ToTensor(),
    transforms.Normalize([0.4886, 0.4886, 0.4760], [0.1482, 0.1395, 0.1342]) #taken these values from: mean_train, std_train = get_mean_std(train_dataloader)
])

train_dataset_norm = CustomDataset(train_dir, labels, transform = train_transforms_norm)
len(train_dataset_norm)

4200

In [7]:
test_transforms_norm = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.ToTensor(),
    transforms.Normalize([0.4670, 0.4365, 0.4319], [0.1659, 0.1498, 0.1380])
])

# dataset = datasets.ImageFolder(data_dir, transform=data_transforms)

test_dataset_norm = CustomDataset(test_dir, labels, transform = test_transforms_norm) 
len(test_dataset_norm)

1346

In [8]:
train_dataloader_norm = torch.utils.data.DataLoader(dataset=train_dataset_norm, batch_size=32, shuffle=True) #takes an image not a tensor 
len(train_dataloader_norm)

132

In [9]:
test_dataloader_norm = torch.utils.data.DataLoader(dataset=test_dataset_norm, batch_size=32, shuffle=True) #takes an image not a tensor 
len(test_dataloader_norm)

43

In [10]:
class_names = train_dataset.classes
class_names

['not_car', 'car']

In [11]:
train_dataset_norm = CustomDataset(train_dir, labels, transform = train_transforms_norm) 

In [12]:
from torchvision.models import resnet18
# Load the pre-trained ResNet18 model
model = resnet18(pretrained=True)
num_classes = len(train_dataset.classes)

C:\Users\sudur\miniconda3\envs\DeepLearning\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\sudur\miniconda3\envs\DeepLearning\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
# Freeze all layers except the final classification layer
for name, param in model.named_parameters():
    if "fc" in name:  # Unfreeze the final classification layer
        param.requires_grad = True
    else:
        param.requires_grad = False

In [14]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.0001)  # Use all parameters

In [15]:
# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [16]:
# Training loop
num_epochs = 40
batch_size = 32

best_accuracy=0.0

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 50)
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    total_samples = 0.0
    
    for i, (images,labels) in enumerate(train_dataloader_norm):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
                            
        optimizer.zero_grad()
                            
        outputs=model(images)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        total_samples += labels.size(0)
        
        # Print mini-batch statistics
        if (i + 1) % 50 == 0:
            mini_batch_loss = train_loss / total_samples
            mini_batch_accuracy = train_accuracy / total_samples
            print(f"Minibatch Loss: {mini_batch_loss:.4f}  Accuracy: {mini_batch_accuracy:.4f}")
    
    # Calculate average loss and accuracy for the epoch
    epoch_loss = train_loss / len(train_dataloader_norm.dataset)
    epoch_accuracy = train_accuracy / len(train_dataloader_norm.dataset)
    
    # Evaluation on testing dataset
    model.eval()
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_dataloader_norm):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    test_accuracy=test_accuracy/len(test_dataloader_norm.dataset)
    
    print(f"Loss: {epoch_loss:.4f}  Accuracy: {epoch_accuracy:.4f}")
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'resnet18_checkpoint.model')
        best_accuracy=test_accuracy

Epoch 1/40
--------------------------------------------------
Minibatch Loss: 7.7711  Accuracy: 0.0169
Minibatch Loss: 5.9939  Accuracy: 0.0991
Loss: 5.1470  Accuracy: 0.1598
Epoch 2/40
--------------------------------------------------
Minibatch Loss: 1.6311  Accuracy: 0.4738
Minibatch Loss: 1.4304  Accuracy: 0.5088
Loss: 1.3379  Accuracy: 0.5348
Epoch 3/40
--------------------------------------------------
Minibatch Loss: 0.9385  Accuracy: 0.6288
Minibatch Loss: 0.9018  Accuracy: 0.6312
Loss: 0.8637  Accuracy: 0.6448
Epoch 4/40
--------------------------------------------------
Minibatch Loss: 0.7415  Accuracy: 0.6875
Minibatch Loss: 0.6996  Accuracy: 0.6987
Loss: 0.6854  Accuracy: 0.7076
Epoch 5/40
--------------------------------------------------
Minibatch Loss: 0.5743  Accuracy: 0.7688
Minibatch Loss: 0.5584  Accuracy: 0.7594
Loss: 0.5573  Accuracy: 0.7595
Epoch 6/40
--------------------------------------------------
Minibatch Loss: 0.4782  Accuracy: 0.7987
Minibatch Loss: 0.4656

## References
i. https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

ii. https://github.com/AarohiSingla/Image-Classification-Using-Pytorch/blob/main/image_classification.ipynb